# Assignment 3

### First import all the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

# Load datasets

### Load Test Data

In [2]:
df_houseprices_testdata = pd.read_csv('test.csv', sep=',')


### Load Train Data

In [3]:
df_houseprices_traindata = pd.read_csv('train.csv', sep=',')
df_houseprices_traindata.set_index(["Id"]).head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Select Columns

In [44]:
fit_cols_num = ["LotArea","OverallQual","OverallCond"]

Now I need to figure out what to focus on, I might want the fields:
    - (YearRemodAdd - YrSold)*-1 = delta_years_mod_and_sold
    - LotArea OK
    - OverallCond OK
    - OverallQual OK
    - MSZoning OK

## Numeric Columns Select

Now that we have separated them into coloumns, we can select the other Quality, condition and LotArea coloumns. 
Then we join them together, we now just need the last coloumn; "delta_years_mod_and_sold"

In [59]:
df_numeric_cols = pd.DataFrame(df_houseprices_testdata[fit_cols_num])
df_numeric_cols.head()

,LotArea,OverallQual,OverallCond
0,11622,5,6
1,14267,6,6
2,13830,5,5
3,9978,6,6
4,5005,8,5


## MSZoning analysis

In [6]:
df_houseprices_traindata["MSZoning"].describe()

count     1460
unique       5
top         RL
freq      1151
Name: MSZoning, dtype: object

It is obvious that there is many low density residential zoned houses, I'll now separate the different MSZones to their own coloumn

In [7]:
MSZones_seperated = pd.get_dummies(df_houseprices_traindata["MSZoning"], prefix="MSZone", drop_first=True, dtype=int)
MSZones_seperated.head()

,MSZone_FV,MSZone_RH,MSZone_RL,MSZone_RM
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


## Join num_cols with MSZones_seperated

In [8]:
#df_numeric_cols = df_numeric_cols.join(MSZones_seperated)
#df_numeric_cols.head()

## Calculate delta years = yr_sold to yr_mod

We calculate the delta_years_mod_and_sold like so: (YearRemodAdd - YrSold)*-1

In [9]:
yearRemodAdd = df_houseprices_traindata["YearRemodAdd"]
yearsold = df_houseprices_traindata["YrSold"]
calculatedCols = (yearRemodAdd-yearsold) * -1
df_delta_years_mod_and_sold = pd.DataFrame(calculatedCols,columns=["delta_years_mod_and_sold"])
df_delta_years_mod_and_sold.head()

,delta_years_mod_and_sold
0,5
1,31
2,6
3,36
4,8


## Join num_cols with Delta years

In [10]:
#df_numeric_cols = df_numeric_cols.join(df_delta_years_mod_and_sold)
#df_numeric_cols.head()

# LinearRegression

In [57]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

a = 40

#instanciate
linReg = LinearRegression()
lasso = Lasso(alpha = a)
ridge = Ridge(alpha = a)


#define X and y
X = df_numeric_cols
y = df_houseprices_traindata.SalePrice

#fit
linReg.fit(X,y)
lasso.fit(X, y)
ridge.fit(X, y)

Ridge(alpha=40, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [60]:
#Predict
#y_predict = linReg.predict(X)
#y_predict = lasso.predict(X)
y_predict = ridge.predict(X)

# to dataframe
df_y_predict = pd.DataFrame(y_predict, columns=["SalePrice"])



# test for negative values
if(np.any(df_y_predict[df_y_predict.SalePrice < 0])):
    print(df_y_predict[df_y_predict.SalePrice < 0])
    

else:
    print("No negative values")

         SalePrice
375  -40217.260602
533  -48515.051346
636   -3220.772844
1100   -863.465339


In [ ]:
ridge = Ridge

In [ ]:
lasso = Lasso

#### For analyzing if values are negative

In [12]:
#df_houseprices_traindata[fit_cols_num].iloc[[88,375,533,636,916,1100]]

In [13]:
#df_houseprices_traindata[fit_cols_num].describe().T

## Mean Sq Error

In [49]:
from sklearn import metrics
# df_y_predict = np.abs(df_y_predict["SalePrice"])
price_log_real = np.log(df_houseprices_traindata.SalePrice)
price_log_predict = np.log(df_y_predict)


metrics.mean_squared_error(price_log_real, price_log_predict)


0.07070623023686344

# to CSV

In [51]:
df_y_predict.head()

0    223539.474521
1    184277.362117
2    227009.040146
3    191694.145363
4    270346.378595
Name: SalePrice, dtype: float64

In [64]:
sub = pd.read_csv('sample_submission.csv')
sub.SalePrice = df_y_predict
sub.to_csv('submission.csv', index=False)

### Notes:

That was.. Bad ... we think?
Let's try remove some columns and see what that does to our prediction quality
- Remove delta year: got worse!
- Remove delta year and MSZones: Worse!
- Add Salecondition: Slightly better!
- Add Neighboorhood: A lot better!

# Testing Ridge and Lasso

In [ ]:
from sklearn.linear_model import Ridge





# I did some testing, tried to see what columns had a numeric value

In [ ]:
# df_dataset_ridge.dtypes != 'object'